In [29]:
import numpy as np
from scipy.optimize import curve_fit
from numpy import sqrt

In [30]:
class DistributionModel:
    def __init__(self):
        ...
    def carbon_number(self,z,A,B):
        """Carbon number fraction function 

        Estimates the carbon number based on a composition
        and two fit  parameters
        Use the Pedersen's ratio 
        This means that it applies from carbon 6 onward
      
        Parameters
        ----------
        z: float
          Molar fraction of component
        A, B: float
            fit parameters

        Returns 
        -------
        carbon_number: float 
            number of single carbon number 
    
        """
        if A is None and B is None:
            A,B = self.A, self.B
        
        return A + B * np.log(z)

    def molar_fraction(self, carbon_number, A, B):
        """ Molar fraction function

        Estimates the molar fraction based on carbon number
        and two fit  parameters
        Use the Pedersen's ratio 
        This means that it applies from carbon 6 onwards
        
        Parameters
        ----------
        carbon_number: float
            Carbon number function
        A, B: float
            fit parameters

        Returns 
        -------
        molar_fraction: float 
            Molar fraction of a given carbon number fraction 
        """
        if A is None and B is None:
            A,B = self.A, self.B
        
        return np.exp(A+B*carbon_number)

In [31]:
class PedersenModel(DistributionModel):
    def __init__(self):
        ...
    def molar_fraction(self, carbon_number, A, B):
        """Molar fraction function
        
        Estimates the molar fraction based on carbon number
        and two fit parameters
        Use the Pedersen's ratio
        This means that it applies from carbon 6 onwards
        
        Parameters
        ----------
        carbon_number: float
            Carbon number function
        A,B: float
            fit parameters

        Returns
        -------
        molar_fraction: float
            Molar fraction of a given carbon number fraction
        """
        if A is None and B is None:
            A,B = self.A, self.B
      
        return np.exp(A+B*carbon_number)

    def molecular_weight(self, carbon_number):
        """ Molecular weight function

        Estimates the molecular weight based on a 
        carbon number fraction

        Parameters
        ----------
        carbon_number: float
            Carbon number function

        Returns 
        -------
        molecular_weight: float 
            Molecular weight of a given carbon number fraction 

        """
        return carbon_number*14-4
    
    def density(self, carbon_number, L, D):
        """ Densities function

        Estimates the densities from C6 onwards 
        based on increase with carbon number

        Parameters
        ----------
        carbon_number: float
            Carbon number function
        L, D: float
            fit parameters

        Returns 
        -------
        density: float 
            Density of a given carbon number fraction

        """
        if L is None and D is None:
            L = self.L
            D = self.D

        return L + D*np.log(carbon_number)


In [32]:
class CismondiModel(DistributionModel):
    def __init__(self):
        ...
    
    def molar_fraction(self, carbon_number, Ac, Bc):
        """ Molar fraction function

        Estimates the molar fraction based on carbon number
        and two fit  parameters
        
        Parameters
        ----------
        carbon_number: float
          Carbon number function
        Ac, Bc: float
          fit parameters

        Returns 
        -------
        molar_fraction: float 
          Molar fraction of a given carbon number fraction 
        """

    def molecular_weight(self,carbon_number,C):
    
        """ Molecular weight function

        Estimates the molecular weight based on carbon number
        and the third fit parameter of Cismondi  "C"

        Parameters
        ----------
        carbon_number: float
            Carbon number function
        c: float
            fit parameter (third parameter of cismondi et al.)

        Returns 
        -------
        molecular_weight: float 
            Molecular weight of a given carbon number fraction
        """

        if C is None:
            C = self.C

        return 84 + C*(carbon_number-6)

    def density(self,carbon_number,Ad):
        """ Density function
        Estimates the densities from C6 onwards 
        based on [...]
      
        Parameters 
        ----------
        carbon_number: float
            Carbon number function
        Ad: float
            fit parameters of cismondi's propose

        Returns
        -------
        density: float
            Density of a given carbon number fraction
        """

        if Ad is None:
            Ad = self.Ad

        Bd = 0.685 - Ad*np.exp(-0.6)
        return Ad*(np.exp(-carbon_number/10))+Bd

In [33]:
class General_Fit(DistributionModel):
    def __init__(self):
        ...
    def fit (self, x, y):
        """ Parameter setting function

        Parameters
        ----------
        x: float
            carbon number
        y: float
            molar fraction

        Return
        ------
        A,B: float
            fit parameters
        """
        self.A, self.B = curve_fit(self.carbon_number, x,y)[0]
        
class Pedersen_Fit(PedersenModel):
    def __init__(self):
        ...
    def fit(self,x):
        """ Parameter setting function

        Parameters
        ----------
        x: float
            carbon number

        Return
        ------
        L,M: float
            fit parameters
        """ 
        self.L, self.M = curve_fit(self.density, x)[0]

class Cismondi_Fit(CismondiModel):
    def __init__(self):
        ...
    def fit(self,x):
        """ Parameter setting function

        Parameters
        ----------
        x: float
            carbon number
        
        Returns
        -------
        C: float
            Third parameter of Cismondi's model
        """ 
        self.C = curve_fit(self.molecular_weight,x)[0]
        self.Ac, self.Bc = curve_fit(self.molar_fraction, x)[0]
        self.Ad = curve_fit(self.density,x)[0]

In [35]:
class Correlations:
    def __init__(self):
        ...
    def critical_temperature(self, molecular_weight,density,c1,c2,c3,c4):
        """Critical temperature correlation
        
        Parameters
        ----------
        molecular_weight: float
            Molecular weight function
        density: float
            Pedersen density
        c1,c2,c3,c4: float
            fit parameters

        Return
        ------
        critical_temperature: float
            Critical temperature
        """
        if c1 is None or c2 is None or c3 is None or c4 is None:
            c1 = self.c1
            c2 = self.c2
            c3 = self.c3
            c4 = self.c4
        
        return c1*density+c2*(np.log(molecular_weight))+c3*molecular_weight*(c4/molecular_weight)

    def critical_pression(self, molecular_weight, density, d1, d2, d3, d4, d5):
        """Critical pression correlation
        
        Parameters
        ----------
        molecular_weight: float
            Molecular weight function
        d1,d2,d3,d4,d5: float
            fit parameters

        Return
        ------
        critical_pression: float
            Critical pression
        """
        if {d1,d2,d3,d4,d5} is None:
            d1 = self.d1
            d2 = self.d2
            d3 = self.d3
            d4 = self.d4
            d5 = self.d5

        pression_log = d1 + d2*(density*(np.exp(d5)+(d3/molecular_weight)+(d4/(molecular_weight)*np.exp(2))))
    
        return np.exp(pression_log)
    
    def m_factor(self, molecular_weight, density, e1, e2, e3, e4):
        """ *m* factor (links to acentric factor as appropiate)
        
        Parameters
        ----------
        molecular_weight: float
            Molecular weight function
        e1,e2,e3,e4: float
            fit parameters

        Return
        ------
        m_factor: float
            m factor to calculate the acentric factor
        """
        if {e1,e2,e3,e4} is None:
            e1 = self.e1
            e2 = self.e2
            e3 = self.e3
            e4 = self.e4
        
        return e1+e2*molecular_weight+e3*density+e4*(molecular_weight*(np.exp(2)))

    def accentric_factor(self,m_factor):
        """ Accentric factor function
        
        Parameters
        ----------
        m_factor: float
            m factor --> links to acentric factor as appropiate
        e1,e2,e3,e4: float
            fit parameters

        Return
        ------
        accentric_factor: float
            Accentric factor
        """
        A = float(0.37464-m_factor)
        B = float(1.54226)
        C = float(-0.26992)
        o1 = (-B + sqrt(B*np.exp(2)-4*A*C))/(2*A)
        o2 = (-B - sqrt(B*np.exp(2)-4*A*C))/(2*A)

        return o2